In [ ]:
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 6.4 MB/s eta 0:00:00


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from catboost import CatBoostClassifier, cv
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
import joblib

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
path_to_data = "/content/drive/MyDrive/SBER_Base/Data"

In [ ]:
path_to_test = os.path.join(path_to_data, "test_final.csv")

X_test = pd.read_csv(path_to_pr_test)

In [ ]:
submission_pd = pd.read_csv(os.path.join(path_to_data, "sample_submission.csv"), index_col="customer_id")

In [ ]:
path_to_df_test = os.path.join(path_to_data, "invest_test_public.csv")

test_df = pd.read_csv(path_to_df_test, index_col='customer_id')

In [ ]:
X_test = pd.read_csv(path_to_test)

In [ ]:
model_path = '/content/drive/MyDrive/SBER_Base/models'

catboost_loaded = joblib.load(os.path.join(model_path, 'catboost_model.pkl'))
rf1_loaded = joblib.load(os.path.join(model_path, 'random_forest_model1.pkl'))
rf2_loaded = joblib.load(os.path.join(model_path, 'random_forest_model2.pkl'))
logreg_loaded = joblib.load(os.path.join(model_path, 'logreg_model.pkl'))
svm_loaded = joblib.load(os.path.join(model_path, 'svm_model.pkl'))
meta_loaded = joblib.load(os.path.join(model_path, 'meta_model.pkl'))
meta_SVM_loaded = joblib.load(os.path.join(model_path, 'meta_model_SVM.pkl'))

meta_XG_loaded = joblib.load(os.path.join(model_path, 'meta_model_XG.pkl'))

/usr/lib/python3.12/pickle.py:1760: UserWarning: [06:39:40] WARNING: /workspace/src/collective/../data/../common/error_msg.h:83: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  setstate(state)


In [ ]:
cat_features = ["risk_profile", "marketing_channel", "membership_tier"]

In [ ]:
seed = 42

###Предсказание

###Стейкинг 1

In [ ]:
boost_proba_test = catboost_loaded.predict_proba(test_df)[:, 1]
rf1_proba_test = rf1_loaded.predict_proba(X_test)[:, 1]
logreg_proba_test = logreg_loaded.predict_proba(X_test)[:, 1]

test_predictions = np.column_stack([boost_proba_test, rf1_proba_test, logreg_proba_test])

final_proba_test = meta_loaded.predict_proba(test_predictions)[:, 1]
y_pred_test = (final_proba_test > 0.5).astype(int)

###Стейкинг 2


In [ ]:
boost_proba_test = catboost_loaded.predict_proba(test_df)[:, 1]
rf1_proba_test = rf1_loaded.predict_proba(X_test)[:, 1]
rf2_proba_test = rf2_loaded.predict_proba(X_test)[:, 1]

test_predictions = np.column_stack([boost_proba_test, rf1_proba_test, rf2_proba_test])

final_proba_test = meta_loaded.predict_proba(test_predictions)[:, 1]
y_pred_test = (final_proba_test > 0.5).astype(int)

###SVM

In [ ]:
svm_proba_test = svm_loaded.predict_proba(X_test)[:, 1]

y_pred_test = (svm_proba_test > 0.5).astype(int)

###SVM + XGboost

In [ ]:
proba_test = pd.DataFrame(svm_loaded.predict_proba(X_test),
                         columns=['prob_class_0', 'prob_class_1'])

X_test_xg = pd.concat([X_test.reset_index(drop=True), proba_test], axis=1)

y_pred_test = meta_XG_loaded.predict(X_test_xg)

###Сохранение

In [ ]:
submission_pd['accepted'] = y_pred_test

In [ ]:
folder_path = '/content/drive/MyDrive/SBER_Base'

os.makedirs(folder_path, exist_ok=True)

path_to_save = os.path.join(folder_path, f"submission_{seed}.csv")
submission_pd.to_csv(path_to_save)

print(f"Файл сохранен: {path_to_save}")

Файл сохранен: /content/drive/MyDrive/SBER_Base/submission_42_boost.csv


In [ ]:
submission_pd

,accepted
customer_id,
6000,1
6002,1
6004,1
6006,1
6008,1
...,...
7990,1
7992,0
7994,1
